<a href="https://colab.research.google.com/github/ReAlex1902/stock-trend-pred/blob/main/get_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install feedparser
!pip install dateparser
!pip install trafilatura

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6048 sha256=dc72882ba95f8a4012bad6d5bee10dbdd80bb96261ed409848225cd63bbf8965
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 kB 30.0 MB/s eta 0:00:00


In [1]:
from pygooglenews import GoogleNews
from bs4 import BeautifulSoup
import json
import numpy as np
import requests
from requests.models import MissingSchema
import spacy
import trafilatura
import time
import pandas as pd

In [2]:
from datetime import datetime, timedelta


def beautifulsoup_extract_text_fallback(response_content):

    '''
    This is a fallback function, so that we can always return a value for text content.
    Even for when both Trafilatura and BeautifulSoup are unable to extract the text from a
    single URL.
    '''

    # Create the beautifulsoup object:
    soup = BeautifulSoup(response_content, 'html.parser')

    # Finding the text:
    text = soup.find_all(text=True)

    # Remove unwanted tag elements:
    cleaned_text = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head',
        'input',
        'script',
        'style',]

    # Then we will loop over every item in the extract text and make sure that the beautifulsoup4 tag
    # is NOT in the blacklist
    for item in text:
        if item.parent.name not in blacklist:
            cleaned_text += '{} '.format(item)

    # Remove any tab separation and strip the text:
    cleaned_text = cleaned_text.replace('\t', '')
    return cleaned_text.strip()


def extract_text_from_single_web_page(url):

    downloaded_url = trafilatura.fetch_url(url)
    try:
        a = trafilatura.extract(downloaded_url, with_metadata=True, include_comments = False, output_format = 'json',
                            date_extraction_params={'extensive_search': True, 'original_date': True})
    except AttributeError:
        a = trafilatura.extract(downloaded_url, with_metadata=True, output_format = 'json',
                            date_extraction_params={'extensive_search': True, 'original_date': True})
    if a:
        json_output = json.loads(a)
        return json_output['text']
    else:
        try:
            resp = requests.get(url)
            # We will only extract the text from successful requests:
            if resp.status_code == 200:
                return beautifulsoup_extract_text_fallback(resp.content)
            else:
                # This line will handle for any failures in both the Trafilature and BeautifulSoup4 functions:
                return np.nan
        # Handling for any URLs that don't have the correct protocol
        except MissingSchema:
            return np.nan


def get_all_dates_between(start_date_str, end_date_str):
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

    current_date = start_date
    date_list = []

    while current_date <= end_date:
        date_list.append(current_date.strftime('%Y-%m-%d'))
        current_date += timedelta(days=1)

    return date_list


def get_news_between_dates(api, dates, topic):
    news_list = []

    for i in range(len(dates)-1):
        from_date = dates[i]
        to_date = dates[i+1]

        result = api.search(topic, from_=from_date, to_=to_date)
        news_list.append(result)

    return news_list


# Пример использования:
gn = GoogleNews()

topic = 'Microsoft'
start_date_str = '2023-01-01'
end_date_str = '2023-02-01'

date_list = get_all_dates_between(start_date_str, end_date_str)
news_result = get_news_between_dates(gn, date_list, topic)

In [20]:
# import requests
# from requests.adapters import HTTPAdapter
# from urllib3.util.retry import Retry

# url = 'https://news.google.com/rss/articles/CBMiQWh0dHBzOi8vd3d3LnRlY2hyZXB1YmxpYy5jb20vYXJ0aWNsZS9taWNyb3NvZnQtaW52ZXN0bWVudC1vcGVuYWkv0gEA?oc=5'

# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',}

# r = requests.post(url, headers=headers)
# r.raise_for_status()

In [9]:
try:
    with open('used-google-links.txt') as myfile:
        used_links = myfile.readlines()
        for i in range(len(used_links)):
            used_links[i] = used_links[i].split()
    used_links_df = pd.DataFrame(used_links, columns = ['id', 'id_short', 'google-link', 'link', 'date'])
except:
    used_links_df = pd.DataFrame(columns = ['id', 'id_short', 'google-link', 'link', 'date'])


for day_idx in range(len(news_result)):
    day_news = news_result[day_idx]['entries']

    for news_idx in range(len(day_news)):
        news = day_news[news_idx]
        google_news_link = news['link']
        news_date = time.strftime('%Y-%m-%d', news['published_parsed'])
        title = news['title']
        id = news['id']
        id_short = id[:250]

        if id not in used_links_df['id'].tolist() and \
           google_news_link not in used_links_df['google-link'].tolist():

            headers = requests.utils.default_headers()

            headers.update(
                {
                    'User-Agent': 'My User Agent 2.0',
                }
            )
            news_url = requests.get(google_news_link, headers=headers).url
            text = extract_text_from_single_web_page(url=news_url)

            print(google_news_link)
            print(news_url)
            print(news_date)

            with open(f"/content/msft-texts/{id_short}.txt", 'a') as myfile:
                myfile.write(id)
                myfile.write("\n")
                myfile.write(title)
                myfile.write("\n")
                myfile.write(text)

            with open("used-google-links.txt", "a") as myfile:
                myfile.write(id)
                myfile.write(" ")
                myfile.write(id_short)
                myfile.write(" ")
                myfile.write(google_news_link)
                myfile.write(" ")
                myfile.write(news_url)
                myfile.write(" ")
                myfile.write(news_date)
                myfile.write("\n")

            new_row = {
                'id': id,
                'id_short': id_short,
                'google-link': google_news_link,
                'link': news_url,
                'date': news_date
            }
            used_links_df = pd.concat([used_links_df, pd.Series(new_row).to_frame().T], ignore_index = True)

        # break
    # break

ERROR:trafilatura.downloads:download error: https://news.google.com/rss/articles/CBMiQWh0dHBzOi8vd3d3LnRlY2hyZXB1YmxpYy5jb20vYXJ0aWNsZS9taWNyb3NvZnQtaW52ZXN0bWVudC1vcGVuYWkv0gEA?oc=5 HTTPSConnectionPool(host='news.google.com', port=443): Max retries exceeded with url: /rss/articles/CBMiQWh0dHBzOi8vd3d3LnRlY2hyZXB1YmxpYy5jb20vYXJ0aWNsZS9taWNyb3NvZnQtaW52ZXN0bWVudC1vcGVuYWkv0gEA?oc=5 (Caused by ResponseError('too many 503 error responses'))


https://news.google.com/rss/articles/CBMiQWh0dHBzOi8vd3d3LnRlY2hyZXB1YmxpYy5jb20vYXJ0aWNsZS9taWNyb3NvZnQtaW52ZXN0bWVudC1vcGVuYWkv0gEA?oc=5
https://news.google.com/rss/articles/CBMiQWh0dHBzOi8vd3d3LnRlY2hyZXB1YmxpYy5jb20vYXJ0aWNsZS9taWNyb3NvZnQtaW52ZXN0bWVudC1vcGVuYWkv0gEA?oc=5
2023-01-24


TypeError: ignored

In [23]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [29]:
!cp -r /content/msft-texts /content/gdrive/Shareddrives/gdrive/datasets/thesis
# !cp used-google-links.txt /content/gdrive/Shareddrives/gdrive/datasets/thesis